In [2]:
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [3]:
# Carregando os dados
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df = pd.concat([df_train, df_test], sort=False)

# Dividindo df_train em conjunto de treino e validação
train_data, val_data = train_test_split(df_train, test_size=0.2, random_state=42)

# Separando features (X) e target (y) para treino, validação e teste
x_train = train_data.drop('target', axis=1)
y_train = train_data['target']

x_val = val_data.drop('target', axis=1)
y_val = val_data['target']

x_test = df_test.drop('target', axis=1)
y_test = df_test['target']

# Aplicando LabelEncoder para transformar a variável alvo em numérica
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_val_encoded = le.transform(y_val)
y_test_encoded = le.transform(y_test)

# Padronizando as variáveis independentes
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

In [10]:
from sklearn.ensemble import RandomForestClassifier
param_dist_rf = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

from sklearn.neural_network import MLPClassifier
param_dist_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],  # Tamanhos das camadas ocultas
    'activation': ['tanh', 'relu', 'logistic'],  # Funções de ativação
    'solver': ['adam', 'sgd'],  # Otimizadores
    'alpha': [0.0001, 0.001, 0.01],  # Regularização L2
    'learning_rate': ['constant', 'adaptive'],  # Taxa de aprendizado
    'max_iter': [200, 300, 500]  # Número máximo de iterações
}

from xgboost import XGBClassifier

param_dist_xgb = {
    'n_estimators': [50, 100, 200, 300],  # Número de árvores
    'max_depth': [3, 5, 7, 10],  # Profundidade máxima
    'learning_rate': [0.01, 0.1, 0.3],  # Taxa de aprendizado
    'subsample': [0.6, 0.8, 1.0],  # Proporção de amostras por árvore
    'colsample_bytree': [0.6, 0.8, 1.0],  # Proporção de colunas por árvore
    'gamma': [0, 0.1, 0.5],  # Poda mínima de ganho
    'reg_alpha': [0, 0.1, 1.0],  # Regularização L1
    'reg_lambda': [0, 1.0, 10.0]  # Regularização L2
}

from lightgbm import LGBMClassifier

param_dist_lgbm = {
    'n_estimators': [50, 100, 200, 300],  # Número de árvores
    'max_depth': [-1, 10, 20, 30],  # -1 significa sem limite
    'learning_rate': [0.01, 0.1, 0.3],  # Taxa de aprendizado
    'num_leaves': [31, 50, 70],  # Número de folhas por árvore
    'subsample': [0.6, 0.8, 1.0],  # Proporção de amostras
    'colsample_bytree': [0.6, 0.8, 1.0],  # Proporção de colunas
    'reg_alpha': [0, 0.1, 1.0],  # Regularização L1
    'reg_lambda': [0, 1.0, 10.0]  # Regularização L2
}

from sklearn.svm import SVC

param_dist_svm = {
    'C': [0.1, 1.0, 2.5, 10.0],  # Parâmetro de regularização
    'kernel': ['linear', 'rbf', 'poly'],  # Tipos de kernel
    'gamma': ['scale', 'auto', 0.1, 1.0],  # Parâmetro do kernel
    'degree': [2, 3, 4],  # Grau do kernel polinomial (se 'poly')
    'class_weight': [None, 'balanced']  # Peso das classes
}

from sklearn.neighbors import KNeighborsClassifier

param_dist_knn = {
    'n_neighbors': [3, 5, 7, 10, 15],  # Número de vizinhos
    'weights': ['uniform', 'distance'],  # Peso dos vizinhos
    'algorithm': ['auto', 'ball_tree', 'kd_tree'],  # Algoritmo de busca
    'p': [1, 2]  # Distância (1 = Manhattan, 2 = Euclidiana)
}

from sklearn.tree import DecisionTreeClassifier

param_dist_dt = {
    'max_depth': [None, 10, 20, 30],  # Profundidade máxima
    'min_samples_split': [2, 5, 10],  # Mínimo de amostras para split
    'min_samples_leaf': [1, 2, 4],  # Mínimo de amostras por folha
    'criterion': ['gini', 'entropy'],  # Critério de divisão
    'class_weight': [None, 'balanced']  # Peso das classes
}

from sklearn.ensemble import GradientBoostingClassifier

param_dist_gb = {
    'n_estimators': [50, 100, 200, 300],  # Número de árvores
    'max_depth': [3, 5, 7],  # Profundidade máxima
    'learning_rate': [0.01, 0.1, 0.3],  # Taxa de aprendizado
    'subsample': [0.6, 0.8, 1.0],  # Proporção de amostras
    'min_samples_split': [2, 5, 10],  # Mínimo de amostras para split
    'min_samples_leaf': [1, 2, 4]  # Mínimo de amostras por folha
}


In [ ]:
# Listas para armazenar resultados
results = []

# Loop de 20 iterações
for i in range(20):
    print(f"Iteração {i+1}/20")
    
    # Inicializando o modelo base
    rf = RandomForestClassifier(random_state=42)
    
    # Configurando o RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist_rf,
        n_iter=10,  # 10 combinações de hiperparâmetros por iteração
        cv=10,      # Validação cruzada com 10 folds
        scoring='f1_macro',  # Métrica de avaliação
        random_state=42,      # Variação no random_state para diversidade
        n_jobs=-1            # Usar todos os núcleos disponíveis
    )
    
    # Treinando o modelo com RandomizedSearchCV
    random_search.fit(x_train_scaled, y_train_encoded)
    
    # Melhor modelo da iteração
    best_model = random_search.best_estimator_
    
    # Previsões no conjunto de treino e validação
    y_train_pred = best_model.predict(x_train_scaled)
    y_val_pred = best_model.predict(x_val_scaled)
    
    # Calculando métricas de treino
    train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
    train_precision = precision_score(y_train_encoded, y_train_pred, average='macro')
    train_recall = recall_score(y_train_encoded, y_train_pred, average='macro')
    train_f1 = f1_score(y_train_encoded, y_train_pred, average='macro')
    
    # Calculando métricas de validação
    val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
    val_precision = precision_score(y_val_encoded, y_val_pred, average='macro')
    val_recall = recall_score(y_val_encoded, y_val_pred, average='macro')
    val_f1 = f1_score(y_val_encoded, y_val_pred, average='macro')
    
    # Armazenando os resultados da iteração
    results.append({
        'iteration': i+1,
        'params': random_search.best_params_,
        'train_accuracy': train_accuracy,
        'train_precision': train_precision,
        'train_recall': train_recall,
        'train_f1': train_f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })

# Convertendo resultados para DataFrame
results_df = pd.DataFrame(results)

# Encontrando a configuração com o maior f1_score de validação
best_iteration = results_df.loc[results_df['val_f1'].idxmax()]
best_params = best_iteration['params']
print("\nMelhor configuração encontrada:")
print(best_params)

# Treinando o modelo final com a melhor configuração
final_model = RandomForestClassifier(**best_params, random_state=42)
final_model.fit(x_train_scaled, y_train_encoded)

# Previsões no conjunto de teste
y_test_pred = final_model.predict(x_test_scaled)

# Calculando métricas no conjunto de teste
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average='macro')
test_recall = recall_score(y_test_encoded, y_test_pred, average='macro')
test_f1 = f1_score(y_test_encoded, y_test_pred, average='macro')

# Exibindo os resultados finais
print("\nMétricas no conjunto de teste com o melhor modelo:")
print(f"Acurácia: {test_accuracy:.4f}")
print(f"Precisão (macro): {test_precision:.4f}")
print(f"Recall (macro): {test_recall:.4f}")
print(f"F1-Score (macro): {test_f1:.4f}")

# Opcional: Exibir todos os resultados
print("\nResumo de todas as iterações:")
print(results_df[['iteration', 'val_f1']])

# os resultados de RandomForest em json extraindo da lista results
import json
with open('RandomForest_resultados.json', 'w') as f:
    json.dump(results, f)

In [16]:
# Listas para armazenar resultados
results = []

# Loop de 20 iterações
for i in range(20):
    print(f"Iteração {i+1}/20")
    
    # Inicializando o modelo base
    rf = DecisionTreeClassifier(random_state=42)
    
    # Configurando o RandomizedSearchCV
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist_dt,
        n_iter=10,  # 10 combinações de hiperparâmetros por iteração
        cv=10,      # Validação cruzada com 10 folds
        scoring='f1_macro',  # Métrica de avaliação
        random_state=42,      # Variação no random_state para diversidade
        n_jobs=-1            # Usar todos os núcleos disponíveis
    )
    
    # Treinando o modelo com RandomizedSearchCV
    random_search.fit(x_train_scaled, y_train_encoded)
    
    # Melhor modelo da iteração
    best_model = random_search.best_estimator_
    
    # Previsões no conjunto de treino e validação
    y_train_pred = best_model.predict(x_train_scaled)
    y_val_pred = best_model.predict(x_val_scaled)
    
    # Calculando métricas de treino
    train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
    train_precision = precision_score(y_train_encoded, y_train_pred, average='macro')
    train_recall = recall_score(y_train_encoded, y_train_pred, average='macro')
    train_f1 = f1_score(y_train_encoded, y_train_pred, average='macro')
    
    # Calculando métricas de validação
    val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
    val_precision = precision_score(y_val_encoded, y_val_pred, average='macro')
    val_recall = recall_score(y_val_encoded, y_val_pred, average='macro')
    val_f1 = f1_score(y_val_encoded, y_val_pred, average='macro')
    
    # Armazenando os resultados da iteração
    results.append({
        'iteration': i+1,
        'params': random_search.best_params_,
        'train_accuracy': train_accuracy,
        'train_precision': train_precision,
        'train_recall': train_recall,
        'train_f1': train_f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })

# Convertendo resultados para DataFrame
results_df = pd.DataFrame(results)

# Encontrando a configuração com o maior f1_score de validação
best_iteration = results_df.loc[results_df['val_f1'].idxmax()]
best_params = best_iteration['params']
print("\nMelhor configuração encontrada:")
print(best_params)

# Treinando o modelo final com a melhor configuração
final_model = DecisionTreeClassifier(**best_params, random_state=42)
final_model.fit(x_train_scaled, y_train_encoded)

# Previsões no conjunto de teste
y_test_pred = final_model.predict(x_test_scaled)

# Calculando métricas no conjunto de teste
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average='macro')
test_recall = recall_score(y_test_encoded, y_test_pred, average='macro')
test_f1 = f1_score(y_test_encoded, y_test_pred, average='macro')

# Exibindo os resultados finais
print("\nMétricas no conjunto de teste com o melhor modelo:")
print(f"Acurácia: {test_accuracy:.4f}")
print(f"Precisão (macro): {test_precision:.4f}")
print(f"Recall (macro): {test_recall:.4f}")
print(f"F1-Score (macro): {test_f1:.4f}")

# Opcional: Exibir todos os resultados
print("\nResumo de todas as iterações:")
print(results_df[['iteration', 'val_f1']])

# os resultados de RandomForest em json extraindo da lista results
import json
with open('DecisionTree_resultados.json', 'w') as f:
    json.dump(results, f)

Iteração 1/20
Iteração 2/20
Iteração 3/20
Iteração 4/20
Iteração 5/20
Iteração 6/20
Iteração 7/20
Iteração 8/20
Iteração 9/20
Iteração 10/20
Iteração 11/20
Iteração 12/20
Iteração 13/20
Iteração 14/20
Iteração 15/20
Iteração 16/20
Iteração 17/20
Iteração 18/20
Iteração 19/20
Iteração 20/20

Melhor configuração encontrada:
{'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 20, 'criterion': 'gini', 'class_weight': None}

Métricas no conjunto de teste com o melhor modelo:
Acurácia: 0.8642
Precisão (macro): 0.8643
Recall (macro): 0.8624
F1-Score (macro): 0.8629

Resumo de todas as iterações:
    iteration    val_f1
0           1  0.852598
1           2  0.852598
2           3  0.852598
3           4  0.852598
4           5  0.852598
5           6  0.852598
6           7  0.852598
7           8  0.852598
8           9  0.852598
9          10  0.852598
10         11  0.852598
11         12  0.852598
12         13  0.852598
13         14  0.852598
14         15  0.852598
15         

In [18]:
# Listas para armazenar resultados
results = []

# Loop de 20 iterações
for i in range(20):
    print(f"Iteração {i+1}/20")
    
    rf = KNeighborsClassifier()
    
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist_knn,
        n_iter=10,  # 10 combinações de hiperparâmetros por iteração
        cv=10,      # Validação cruzada com 10 folds
        scoring='f1_macro',  # Métrica de avaliação
        random_state=42,      # Variação no random_state para diversidade
        n_jobs=-1            # Usar todos os núcleos disponíveis
    )
    
    random_search.fit(x_train_scaled, y_train_encoded)
    
    # Melhor modelo da iteração
    best_model = random_search.best_estimator_
    
    # Previsões no conjunto de treino e validação
    y_train_pred = best_model.predict(x_train_scaled)
    y_val_pred = best_model.predict(x_val_scaled)
    
    # Calculando métricas de treino
    train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
    train_precision = precision_score(y_train_encoded, y_train_pred, average='macro')
    train_recall = recall_score(y_train_encoded, y_train_pred, average='macro')
    train_f1 = f1_score(y_train_encoded, y_train_pred, average='macro')
    
    # Calculando métricas de validação
    val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
    val_precision = precision_score(y_val_encoded, y_val_pred, average='macro')
    val_recall = recall_score(y_val_encoded, y_val_pred, average='macro')
    val_f1 = f1_score(y_val_encoded, y_val_pred, average='macro')
    
    # Armazenando os resultados da iteração
    results.append({
        'iteration': i+1,
        'params': random_search.best_params_,
        'train_accuracy': train_accuracy,
        'train_precision': train_precision,
        'train_recall': train_recall,
        'train_f1': train_f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })

# Convertendo resultados para DataFrame
results_df = pd.DataFrame(results)

# Encontrando a configuração com o maior f1_score de validação
best_iteration = results_df.loc[results_df['val_f1'].idxmax()]
best_params = best_iteration['params']
print("\nMelhor configuração encontrada:")
print(best_params)

# Treinando o modelo final com a melhor configuração
final_model = KNeighborsClassifier(**best_params)
final_model.fit(x_train_scaled, y_train_encoded)

# Previsões no conjunto de teste
y_test_pred = final_model.predict(x_test_scaled)

# Calculando métricas no conjunto de teste
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average='macro')
test_recall = recall_score(y_test_encoded, y_test_pred, average='macro')
test_f1 = f1_score(y_test_encoded, y_test_pred, average='macro')

# Exibindo os resultados finais
print("\nMétricas no conjunto de teste com o melhor modelo:")
print(f"Acurácia: {test_accuracy:.4f}")
print(f"Precisão (macro): {test_precision:.4f}")
print(f"Recall (macro): {test_recall:.4f}")
print(f"F1-Score (macro): {test_f1:.4f}")

# Opcional: Exibir todos os resultados
print("\nResumo de todas as iterações:")
print(results_df[['iteration', 'val_f1']])

# os resultados de RandomForest em json extraindo da lista results
import json
with open('KNN_resultados.json', 'w') as f:
    json.dump(results, f)

Iteração 1/20
Iteração 2/20
Iteração 3/20
Iteração 4/20
Iteração 5/20
Iteração 6/20
Iteração 7/20
Iteração 8/20
Iteração 9/20
Iteração 10/20
Iteração 11/20
Iteração 12/20
Iteração 13/20
Iteração 14/20
Iteração 15/20
Iteração 16/20
Iteração 17/20
Iteração 18/20
Iteração 19/20
Iteração 20/20

Melhor configuração encontrada:
{'weights': 'distance', 'p': 1, 'n_neighbors': 5, 'algorithm': 'auto'}

Métricas no conjunto de teste com o melhor modelo:
Acurácia: 0.9470
Precisão (macro): 0.9467
Recall (macro): 0.9460
F1-Score (macro): 0.9461

Resumo de todas as iterações:
    iteration    val_f1
0           1  0.949801
1           2  0.949801
2           3  0.949801
3           4  0.949801
4           5  0.949801
5           6  0.949801
6           7  0.949801
7           8  0.949801
8           9  0.949801
9          10  0.949801
10         11  0.949801
11         12  0.949801
12         13  0.949801
13         14  0.949801
14         15  0.949801
15         16  0.949801
16         17  0.949801


In [ ]:
# Listas para armazenar resultados
results = []

# Loop de 20 iterações
for i in range(20):
    print(f"Iteração {i+1}/20")
    
    rf = MLPClassifier()
    
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist_mlp,
        n_iter=10,  # 10 combinações de hiperparâmetros por iteração
        cv=10,      # Validação cruzada com 10 folds
        scoring='f1_macro',  # Métrica de avaliação
        random_state=42,      # Variação no random_state para diversidade
        n_jobs=-1            # Usar todos os núcleos disponíveis
    )
    
    random_search.fit(x_train_scaled, y_train_encoded)
    
    # Melhor modelo da iteração
    best_model = random_search.best_estimator_
    
    # Previsões no conjunto de treino e validação
    y_train_pred = best_model.predict(x_train_scaled)
    y_val_pred = best_model.predict(x_val_scaled)
    
    # Calculando métricas de treino
    train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
    train_precision = precision_score(y_train_encoded, y_train_pred, average='macro')
    train_recall = recall_score(y_train_encoded, y_train_pred, average='macro')
    train_f1 = f1_score(y_train_encoded, y_train_pred, average='macro')
    
    # Calculando métricas de validação
    val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
    val_precision = precision_score(y_val_encoded, y_val_pred, average='macro')
    val_recall = recall_score(y_val_encoded, y_val_pred, average='macro')
    val_f1 = f1_score(y_val_encoded, y_val_pred, average='macro')
    
    # Armazenando os resultados da iteração
    results.append({
        'iteration': i+1,
        'params': random_search.best_params_,
        'train_accuracy': train_accuracy,
        'train_precision': train_precision,
        'train_recall': train_recall,
        'train_f1': train_f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })

# Convertendo resultados para DataFrame
results_df = pd.DataFrame(results)

# Encontrando a configuração com o maior f1_score de validação
best_iteration = results_df.loc[results_df['val_f1'].idxmax()]
best_params = best_iteration['params']
print("\nMelhor configuração encontrada:")
print(best_params)

# Treinando o modelo final com a melhor configuração
final_model = MLPClassifier(**best_params, random_state=42)
final_model.fit(x_train_scaled, y_train_encoded)

# Previsões no conjunto de teste
y_test_pred = final_model.predict(x_test_scaled)

# Calculando métricas no conjunto de teste
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average='macro')
test_recall = recall_score(y_test_encoded, y_test_pred, average='macro')
test_f1 = f1_score(y_test_encoded, y_test_pred, average='macro')

# Exibindo os resultados finais
print("\nMétricas no conjunto de teste com o melhor modelo:")
print(f"Acurácia: {test_accuracy:.4f}")
print(f"Precisão (macro): {test_precision:.4f}")
print(f"Recall (macro): {test_recall:.4f}")
print(f"F1-Score (macro): {test_f1:.4f}")

# Opcional: Exibir todos os resultados
print("\nResumo de todas as iterações:")
print(results_df[['iteration', 'val_f1']])

# os resultados de RandomForest em json extraindo da lista results
import json
with open('MLP_resultados.json', 'w') as f:
    json.dump(results, f)

Iteração 1/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

Iteração 2/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

Iteração 3/20


...

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

Iteração 11/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

Iteração 12/20


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iteration

In [ ]:
# Listas para armazenar resultados
results = []

# Loop de 20 iterações
for i in range(20):
    print(f"Iteração {i+1}/20")
    
    rf = XGBClassifier()
    
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist_xgb,
        n_iter=10,  # 10 combinações de hiperparâmetros por iteração
        cv=10,      # Validação cruzada com 10 folds
        scoring='f1_macro',  # Métrica de avaliação
        random_state=42,      # Variação no random_state para diversidade
        n_jobs=-1            # Usar todos os núcleos disponíveis
    )
    
    random_search.fit(x_train_scaled, y_train_encoded)
    
    # Melhor modelo da iteração
    best_model = random_search.best_estimator_
    
    # Previsões no conjunto de treino e validação
    y_train_pred = best_model.predict(x_train_scaled)
    y_val_pred = best_model.predict(x_val_scaled)
    
    # Calculando métricas de treino
    train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
    train_precision = precision_score(y_train_encoded, y_train_pred, average='macro')
    train_recall = recall_score(y_train_encoded, y_train_pred, average='macro')
    train_f1 = f1_score(y_train_encoded, y_train_pred, average='macro')
    
    # Calculando métricas de validação
    val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
    val_precision = precision_score(y_val_encoded, y_val_pred, average='macro')
    val_recall = recall_score(y_val_encoded, y_val_pred, average='macro')
    val_f1 = f1_score(y_val_encoded, y_val_pred, average='macro')
    
    # Armazenando os resultados da iteração
    results.append({
        'iteration': i+1,
        'params': random_search.best_params_,
        'train_accuracy': train_accuracy,
        'train_precision': train_precision,
        'train_recall': train_recall,
        'train_f1': train_f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })

# Convertendo resultados para DataFrame
results_df = pd.DataFrame(results)

# Encontrando a configuração com o maior f1_score de validação
best_iteration = results_df.loc[results_df['val_f1'].idxmax()]
best_params = best_iteration['params']
print("\nMelhor configuração encontrada:")
print(best_params)

# Treinando o modelo final com a melhor configuração
final_model = XGBClassifier(**best_params)
final_model.fit(x_train_scaled, y_train_encoded)

# Previsões no conjunto de teste
y_test_pred = final_model.predict(x_test_scaled)

# Calculando métricas no conjunto de teste
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average='macro')
test_recall = recall_score(y_test_encoded, y_test_pred, average='macro')
test_f1 = f1_score(y_test_encoded, y_test_pred, average='macro')

# Exibindo os resultados finais
print("\nMétricas no conjunto de teste com o melhor modelo:")
print(f"Acurácia: {test_accuracy:.4f}")
print(f"Precisão (macro): {test_precision:.4f}")
print(f"Recall (macro): {test_recall:.4f}")
print(f"F1-Score (macro): {test_f1:.4f}")

# Opcional: Exibir todos os resultados
print("\nResumo de todas as iterações:")
print(results_df[['iteration', 'val_f1']])

# os resultados de RandomForest em json extraindo da lista results
import json
with open('XGBoost_resultados.json', 'w') as f:
    json.dump(results, f)

In [ ]:
# Listas para armazenar resultados
results = []

# Loop de 20 iterações
for i in range(20):
    print(f"Iteração {i+1}/20")
    
    rf = LGBMClassifier()
    
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist_xgb,
        n_iter=10,  # 10 combinações de hiperparâmetros por iteração
        cv=10,      # Validação cruzada com 10 folds
        scoring='f1_macro',  # Métrica de avaliação
        random_state=42,      # Variação no random_state para diversidade
        n_jobs=-1            # Usar todos os núcleos disponíveis
    )
    
    random_search.fit(x_train_scaled, y_train_encoded)
    
    # Melhor modelo da iteração
    best_model = random_search.best_estimator_
    
    # Previsões no conjunto de treino e validação
    y_train_pred = best_model.predict(x_train_scaled)
    y_val_pred = best_model.predict(x_val_scaled)
    
    # Calculando métricas de treino
    train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
    train_precision = precision_score(y_train_encoded, y_train_pred, average='macro')
    train_recall = recall_score(y_train_encoded, y_train_pred, average='macro')
    train_f1 = f1_score(y_train_encoded, y_train_pred, average='macro')
    
    # Calculando métricas de validação
    val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
    val_precision = precision_score(y_val_encoded, y_val_pred, average='macro')
    val_recall = recall_score(y_val_encoded, y_val_pred, average='macro')
    val_f1 = f1_score(y_val_encoded, y_val_pred, average='macro')
    
    # Armazenando os resultados da iteração
    results.append({
        'iteration': i+1,
        'params': random_search.best_params_,
        'train_accuracy': train_accuracy,
        'train_precision': train_precision,
        'train_recall': train_recall,
        'train_f1': train_f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })

# Convertendo resultados para DataFrame
results_df = pd.DataFrame(results)

# Encontrando a configuração com o maior f1_score de validação
best_iteration = results_df.loc[results_df['val_f1'].idxmax()]
best_params = best_iteration['params']
print("\nMelhor configuração encontrada:")
print(best_params)

# Treinando o modelo final com a melhor configuração
final_model = LGBMClassifier(**best_params, random_state=42)
final_model.fit(x_train_scaled, y_train_encoded)

# Previsões no conjunto de teste
y_test_pred = final_model.predict(x_test_scaled)

# Calculando métricas no conjunto de teste
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average='macro')
test_recall = recall_score(y_test_encoded, y_test_pred, average='macro')
test_f1 = f1_score(y_test_encoded, y_test_pred, average='macro')

# Exibindo os resultados finais
print("\nMétricas no conjunto de teste com o melhor modelo:")
print(f"Acurácia: {test_accuracy:.4f}")
print(f"Precisão (macro): {test_precision:.4f}")
print(f"Recall (macro): {test_recall:.4f}")
print(f"F1-Score (macro): {test_f1:.4f}")

# Opcional: Exibir todos os resultados
print("\nResumo de todas as iterações:")
print(results_df[['iteration', 'val_f1']])

# os resultados de RandomForest em json extraindo da lista results
import json
with open('LightGBM_resultados.json', 'w') as f:
    json.dump(results, f)

In [ ]:
# Listas para armazenar resultados
results = []

# Loop de 20 iterações
for i in range(20):
    print(f"Iteração {i+1}/20")
    
    rf = SVC()
    
    random_search = RandomizedSearchCV(
        estimator=rf,
        param_distributions=param_dist_svm,
        n_iter=10,  # 10 combinações de hiperparâmetros por iteração
        cv=10,      # Validação cruzada com 10 folds
        scoring='f1_macro',  # Métrica de avaliação
        random_state=42,      # Variação no random_state para diversidade
        n_jobs=-1            # Usar todos os núcleos disponíveis
    )
    
    random_search.fit(x_train_scaled, y_train_encoded)
    
    # Melhor modelo da iteração
    best_model = random_search.best_estimator_
    
    # Previsões no conjunto de treino e validação
    y_train_pred = best_model.predict(x_train_scaled)
    y_val_pred = best_model.predict(x_val_scaled)
    
    # Calculando métricas de treino
    train_accuracy = accuracy_score(y_train_encoded, y_train_pred)
    train_precision = precision_score(y_train_encoded, y_train_pred, average='macro')
    train_recall = recall_score(y_train_encoded, y_train_pred, average='macro')
    train_f1 = f1_score(y_train_encoded, y_train_pred, average='macro')
    
    # Calculando métricas de validação
    val_accuracy = accuracy_score(y_val_encoded, y_val_pred)
    val_precision = precision_score(y_val_encoded, y_val_pred, average='macro')
    val_recall = recall_score(y_val_encoded, y_val_pred, average='macro')
    val_f1 = f1_score(y_val_encoded, y_val_pred, average='macro')
    
    # Armazenando os resultados da iteração
    results.append({
        'iteration': i+1,
        'params': random_search.best_params_,
        'train_accuracy': train_accuracy,
        'train_precision': train_precision,
        'train_recall': train_recall,
        'train_f1': train_f1,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })

# Convertendo resultados para DataFrame
results_df = pd.DataFrame(results)

# Encontrando a configuração com o maior f1_score de validação
best_iteration = results_df.loc[results_df['val_f1'].idxmax()]
best_params = best_iteration['params']
print("\nMelhor configuração encontrada:")
print(best_params)

# Treinando o modelo final com a melhor configuração
final_model = SVC(**best_params)
final_model.fit(x_train_scaled, y_train_encoded)

# Previsões no conjunto de teste
y_test_pred = final_model.predict(x_test_scaled)

# Calculando métricas no conjunto de teste
test_accuracy = accuracy_score(y_test_encoded, y_test_pred)
test_precision = precision_score(y_test_encoded, y_test_pred, average='macro')
test_recall = recall_score(y_test_encoded, y_test_pred, average='macro')
test_f1 = f1_score(y_test_encoded, y_test_pred, average='macro')

# Exibindo os resultados finais
print("\nMétricas no conjunto de teste com o melhor modelo:")
print(f"Acurácia: {test_accuracy:.4f}")
print(f"Precisão (macro): {test_precision:.4f}")
print(f"Recall (macro): {test_recall:.4f}")
print(f"F1-Score (macro): {test_f1:.4f}")

# Opcional: Exibir todos os resultados
print("\nResumo de todas as iterações:")
print(results_df[['iteration', 'val_f1']])

# os resultados de RandomForest em json extraindo da lista results
import json
with open('SVM_resultados.json', 'w') as f:
    json.dump(results, f)